# Parameters

This notebook will walk you through the various ways to interact with parameters in caustics. For each lens and light model there are certain parameters which are given special priority as these parameters are the ones that would be sampled in a simulator. This allows for taking the machinery of caustics and converting it into a function which can use all the power of pytorch, or other sampling/optimization frameworks. 

Throughout the tutorial, keep in mind that parameters are stored in a directed acyclic graph (DAG). This gives a unique way to access each parameter

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from torch.nn.functional import avg_pool2d
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np

import caustics

## Setting static/dynamic parameters

Let's see how we can set static and dynamic parameters. In caustics, a dynamic parameter is one which will be involved in sampling and must be provided on evaluation of a function. A static parameter has a fixed value and so "disappears" from the graph so that you don't need to worry about it anymore.

In [ ]:
# Flat cosmology with all dynamic parameters
cosmo = caustics.cosmology.FlatLambdaCDM(name="cosmo", h0=None, Om0=None)

# SIE lens with q and b as static parameters
lens = caustics.lenses.SIE(cosmology=cosmo, q=0.4, b=1.0)

# Sersic with all dynamic parameters except the sersic index, effective radius, and effective brightness
source = caustics.light.Sersic(name="source", n=2.0, Re=1.0, Ie=1.0)

# Sersic with all dynamic parameters except the x position, position angle, and effective radius
lens_light = caustics.light.Sersic(name="lenslight", x0=0.0, phi=1.3, Re=1.0)

# A simulator which captures all these parameters into a single DAG
sim = caustics.sims.Lens_Source(
    lens=lens, source=source, lens_light=lens_light, pixelscale=0.05, pixels_x=100
)

We can have the simulator print a graph of the DAG from it's perspective. Note that the white boxes are dynamic parameters while the grey boxes are static parameters

In [ ]:
sim.get_graph(True, True)

In [ ]:
# Accessing a parameter and giving it a value will turn it into a static parameter
sim.SIE.phi = 0.4
sim.get_graph(True, True)

In [ ]:
# Accessing a parameter and setting it to None will turn it into a dynamic parameter
sim.lenslight.x0 = None
sim.get_graph(True, True)

In [ ]:
# This also gives us the order of parameters for a vector that can be an input to the sim function
x_tens = torch.tensor(
    [
        1.5,  # z_s
        0.5,  # sie z_l
        0.1,  # sie x0
        -0.1,  # sie y0
        0.7,  # sie cosmo h0
        0.31,  # sie cosmo Om0
        0.0,  # source x0
        0.0,  # source y0
        0.7,  # source q
        1.4,  # source phi
        0.1,  # lenslight x0
        -0.1,  # lenslight y0
        0.6,  # lenslight q
        3.0,  # lenslight n
        1.0,  # lenslight Ie
    ]
)

# alternatively we can construct a dictionary
x_dict = {
    "sim": {
        "z_s": torch.tensor(1.5),
    },
    "SIE": {
        "z_l": torch.tensor(0.5),
        "x0": torch.tensor(0.1),
        "y0": torch.tensor(-0.1),
    },
    "cosmo": {
        "h0": torch.tensor(0.7),
        "Om0": torch.tensor(0.31),
    },
    "source": {
        "x0": torch.tensor(0.0),
        "y0": torch.tensor(0.0),
        "q": torch.tensor(0.7),
        "phi": torch.tensor(1.4),
    },
    "lenslight": {
        "x0": torch.tensor(0.1),
        "y0": torch.tensor(-0.1),
        "q": torch.tensor(0.6),
        "n": torch.tensor(3.0),
        "Ie": torch.tensor(1.0),
    },
}

fig, axarr = plt.subplots(1, 2, figsize=(16, 8))
axarr[0].imshow(
    sim(sim.pack(x_tens), quad_level=3).detach().cpu().numpy(), origin="lower"
)
axarr[0].set_title("Simulator from tensor")
axarr[1].imshow(
    sim(sim.pack(x_dict), quad_level=3).detach().cpu().numpy(), origin="lower"
)
axarr[1].set_title("Simulator from dictionary")
plt.show()

## Manual Inputs

We have now seen the standard `pack` method of passing dynamic parameters to a caustics function/simulator. This is very powerful at scale, but can be tedious to enter by hand while prototyping and doing tests. Now lets see a more manual way to pass parameters to a function. For this lets try getting the exact position of each of the 4 images of the background source.

In [ ]:
# First find the position of each of the images
x, y = lens.forward_raytrace(
    torch.tensor(0.0),  # First three arguments are regular function arguments
    torch.tensor(0.0),
    torch.tensor(1.5),
    z_l=torch.tensor(0.5),  # Next three are kwargs which give the SIE parameters
    x0=torch.tensor(0.1),
    y0=torch.tensor(-0.1),
    cosmo_h0=torch.tensor(
        0.7
    ),  # Next two are parameters needed for "cosmo" and so they are named as such
    cosmo_Om0=torch.tensor(0.31),
    fov=0.05 * 100,  # Next two are kwargs for the "forward_raytrace" method
    n_init=100,
)

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(
    sim(x_tens, quad_level=3).detach().cpu().numpy(),
    extent=(-0.05 * 100 / 2, 0.05 * 100 / 2, -0.05 * 100 / 2, 0.05 * 100 / 2),
    origin="lower",
)
ax.scatter(x.detach().cpu().numpy(), y.detach().cpu().numpy(), color="r")
plt.show()